In [1]:
import helper_functions as hp
import pandas as pd
import numpy as np
import bus_bounds_data as bb_data
from pypower.case118 import case118
from pypower.savecase import savecase
from keras.models import load_model
from egret.models.acpf import solve_acpf
from egret.parsers.matpower_parser import create_ModelData

In [2]:
generated_data = "case118_data3.csv"
X_train, X_test, y_train, y_test, df, slack_bus_data, scaler = hp.pre_process_data_from_file(file_path=generated_data)

In [3]:
previous_model = True
if not previous_model:
    model = hp.train_model(X_train, X_test, y_train, y_test, plot_losses=True)
    model.save("../saved_models/acopf_model6.h5") # Change model number to not overwrite a previous model
else:
    model_path = "../saved_models/acopf_model5.h5"
    model = load_model(model_path)

2022-07-26 14:12:02.740946: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
y_pred = model.predict(X_test)
label_cols = hp.get_label_cols()
y_test_df = pd.DataFrame(data=y_test, columns=label_cols)
y_pred_df = pd.DataFrame(data=y_pred, columns=label_cols)

938/938 [==============================] - 2s 2ms/step


In [5]:
X_train, X_test = hp.unscale_data(X_train, X_test, scaler)

In [6]:
v_min = 0.94
v_max = 1.06

In [7]:
for col, p_bounds_key in zip(label_cols[::2], bb_data.p_bounds.keys()):
    p_min = bb_data.p_bounds[p_bounds_key]['p_min']
    p_max = bb_data.p_bounds[p_bounds_key]['p_max']
    y_test_df[col] = y_test_df[col].apply(lambda x : hp.unparametrize_func(x, p_min, p_max))
    y_pred_df[col] = y_pred_df[col].apply(lambda x : hp.unparametrize_func(x, p_min, p_max))

for col in label_cols[1::2]:
    y_test_df[col] = y_test_df[col].apply(lambda x : hp.unparametrize_func(x, v_min, v_max))
    y_pred_df[col] = y_pred_df[col].apply(lambda x : hp.unparametrize_func(x, v_min, v_max))

In [8]:
hp.evaluate_model(y_test_df, y_pred_df)

Results of Active Power Generation:
MAE : 1.1155068916739554
MSE : 11.61746130809883
RMSE : 3.4084397175392187

Results of Voltage Generation:
MAE : 1.4306968503479113e-05
MSE : 4.462672696605791e-10
RMSE : 2.1125038926841746e-05


In [9]:
bus_gen_nums = hp.get_bus_gen_nums()

In [10]:
pv_bus_p_gen = {"p{}_gen".format(bus_num) : p_gen
                    for bus_num, p_gen in zip(bus_gen_nums, y_pred_df.loc[0, 'p_gen1'::2].values)}

In [11]:
pq_bus_p_load = {"p{}_load".format(bus_num + 1) : p_load
                    for bus_num, p_load in zip(range(118), X_test[0, ::2])}
                    
pq_bus_q_load = {"q{}_load".format(bus_num + 1) : q_load
                    for bus_num, q_load in zip(range(118), X_test[0, 1::2])}

In [12]:
y = {**pv_bus_p_gen, **pq_bus_p_load, **pq_bus_q_load}

In [13]:
unknown_angles = {"Angle_{}".format(str(bus_num + 1)) : 0 for bus_num in range(118)}

del unknown_angles['Angle_69']

In [14]:
unknown_pq_v_mags = {"Voltage_{}".format(str(bus_num + 1)) : 1
                        for bus_num in range(118) if str(bus_num + 1) not in bus_gen_nums}

In [15]:
x = {**unknown_angles, **unknown_pq_v_mags}

In [16]:
gen_bus_idx = [bus_num + 1 for bus_num in range(118) if str(bus_num + 1) in bus_gen_nums]

In [17]:
known_pv_v_mags = {"Voltage_{}".format(bus_num) : y_pred_df.loc[0, "v_gen1"::2][idx]
                        for bus_num, idx in zip(gen_bus_idx, range(len(gen_bus_idx)))}

In [18]:
slack_bus_v = slack_bus_data.values[0][2]
slack_bus_a = slack_bus_data.values[0][3]

In [19]:
V = []
for idx in range(118):
    if idx + 1 == 69:
        V.append([slack_bus_v, slack_bus_a])
    elif str(idx + 1) not in bus_gen_nums:
        V.append([x["Voltage_{}".format(idx + 1)], x["Angle_{}".format(idx + 1)]])
    else:
        V.append([known_pv_v_mags["Voltage_{}".format(idx + 1)], x["Angle_{}".format(idx + 1)]])

In [20]:
case118_dict = case118()

In [21]:
for idx in range(len(case118_dict['bus'])):
    p_key = "p{}_load".format(idx + 1)
    q_key = "q{}_load".format(idx + 1)
    case118_dict['bus'][idx][2] = y[p_key]
    case118_dict['bus'][idx][3] = y[q_key]
    case118_dict['bus'][idx][7] = V[idx][0]
    case118_dict['bus'][idx][8] = V[idx][1]

In [22]:
V_gen = [v_elem[0] for v_elem in V if v_elem[0] != 1]

In [23]:
for idx, bus_num in zip(range(len(case118_dict['gen'])), hp.get_bus_gen_nums(include_slack=True)):
    if idx != 29:
        p_key = "p{}_gen".format(bus_num)
        case118_dict['gen'][idx][1] = y[p_key]
    else:
        case118_dict['gen'][idx][1] = 0
        
    case118_dict['gen'][idx][5] = V_gen[idx]

In [24]:
savecase("new_case", ppc=case118_dict)

'new_case.py'

In [25]:
original_data_path = "../case118_data/case118.txt"
new_data_path = "../generated_data/case118.txt"

with open("new_case.py") as f:
    lines = f.readlines()
    data = lines[12:130]

new_data = []
for line in data:
    line_arr = line.strip()[1:-2].split(",")
    new_string = ""
    for element in line_arr:
        new_string += element.strip()
        new_string += "    "
    
    new_string = new_string[:-4] + ";"
    new_string += "\n"
    new_data.append(new_string)

with open(original_data_path) as f:
    lines = f.readlines()

with open(new_data_path, 'w') as f:
    f.writelines(lines[:23] + new_data + lines[141:])

In [26]:
with open("new_case.py") as f:
    lines = f.readlines()
    data = lines[135:189]

new_data = []
for line in data:
    line_arr = line.strip()[1:-2].split(",")
    new_string = ""
    for element in line_arr:
        new_string += element.strip()
        new_string += "    "
    
    new_string = new_string[:-4] + ";"
    new_string += "\n"
    new_data.append(new_string)

with open(new_data_path) as f:
    lines = f.readlines()

with open(new_data_path, 'w') as f:
    f.writelines(lines[:145] + new_data + lines[199:])

In [27]:
model_data = create_ModelData(new_data_path)

In [28]:
# Hi Jean-Paul, here is where I start using the power flow solver provided by egret
md, results = solve_acpf(model_data, "ipopt", solver_tee=False, return_results=True)

In [29]:
# Here, we verify that the reactive power constraints were not violated, and if they were,
# correct these violations by either setting them to the lower or upper bound

q_constraint_violation = False
print_qg_violations = True
for key in md.data['elements']['generator'].keys():
    q_min = md.data['elements']['generator'][key]['q_min']
    q_max = md.data['elements']['generator'][key]['q_max']
    qg = md.data['elements']['generator'][key]['qg']
    if not q_min <= qg <= q_max:
        q_constraint_violation = True
        if print_qg_violations:
            print("Bus", md.data['elements']['generator'][key]['bus'], ":")
            print("\tReactive Power Generation :", qg)
            print("\t\tViolated Constraint :", q_min, "<=", qg, "<=", q_max)

        md.data['elements']['generator'][key]['qg'] = min(q_max, max(qg, q_min))

Bus 1 :
	Reactive Power Generation : 17.126003190350502
		Violated Constraint : -5.0 <= 17.126003190350502 <= 15.0
Bus 12 :
	Reactive Power Generation : 244.12548647752877
		Violated Constraint : -35.0 <= 244.12548647752877 <= 120.0
Bus 15 :
	Reactive Power Generation : -39.82132378728101
		Violated Constraint : -10.0 <= -39.82132378728101 <= 30.0
Bus 19 :
	Reactive Power Generation : -9.8620982395899
		Violated Constraint : -8.0 <= -9.8620982395899 <= 24.0
Bus 34 :
	Reactive Power Generation : -21.421883384570858
		Violated Constraint : -8.0 <= -21.421883384570858 <= 24.0
Bus 55 :
	Reactive Power Generation : -15.709409836354785
		Violated Constraint : -8.0 <= -15.709409836354785 <= 23.0
Bus 56 :
	Reactive Power Generation : 17.664780397495075
		Violated Constraint : -8.0 <= 17.664780397495075 <= 15.0
Bus 74 :
	Reactive Power Generation : 11.145662290941145
		Violated Constraint : -6.0 <= 11.145662290941145 <= 9.0
Bus 76 :
	Reactive Power Generation : -17.92890523374042
		Violated Con

In [30]:
# This part is just for you to see that the voltage magnitudes at some busses are not meeting the constraint

print_vm_violations = True
if print_vm_violations:
    for key in md.data['elements']['bus'].keys():
        vm = md.data['elements']['bus'][key]['vm']
        if not v_min <= vm <= v_max:
            print("Bus", key, ":")
            print("\tVoltage Magnitude :", vm)
            print("\t\tViolated Constraint :", v_min, "<=", vm, "<=", v_max)

Bus 2 :
	Voltage Magnitude : 0.8870470833040244
		Violated Constraint : 0.94 <= 0.8870470833040244 <= 1.06
Bus 20 :
	Voltage Magnitude : 0.9123340907607944
		Violated Constraint : 0.94 <= 0.9123340907607944 <= 1.06
Bus 21 :
	Voltage Magnitude : 0.8842757939991804
		Violated Constraint : 0.94 <= 0.8842757939991804 <= 1.06
Bus 22 :
	Voltage Magnitude : 0.9054834687485201
		Violated Constraint : 0.94 <= 0.9054834687485201 <= 1.06
Bus 51 :
	Voltage Magnitude : 0.9192213998301139
		Violated Constraint : 0.94 <= 0.9192213998301139 <= 1.06
Bus 52 :
	Voltage Magnitude : 0.9083665008181213
		Violated Constraint : 0.94 <= 0.9083665008181213 <= 1.06
Bus 53 :
	Voltage Magnitude : 0.9212238051694328
		Violated Constraint : 0.94 <= 0.9212238051694328 <= 1.06
Bus 58 :
	Voltage Magnitude : 0.9123965283695922
		Violated Constraint : 0.94 <= 0.9123965283695922 <= 1.06
Bus 86 :
	Voltage Magnitude : 0.9209025513836534
		Violated Constraint : 0.94 <= 0.9209025513836534 <= 1.06


In [31]:
# If they were reactive power violations, solve the power flow again with the corrected values
if q_constraint_violation:
    md, results = solve_acpf(md, "ipopt", solver_tee=False, return_results=True)

In [32]:
# Same as before, we verify that the constraints were not violated. In my case, the solver arrives
# at the same solution even if the reactive power generation setpoints were corrected

for key in md.data['elements']['generator'].keys():
    q_min = md.data['elements']['generator'][key]['q_min']
    q_max = md.data['elements']['generator'][key]['q_max']
    qg = md.data['elements']['generator'][key]['qg']
    if not q_min <= qg <= q_max:
        print("Bus", md.data['elements']['generator'][key]['bus'], ":")
        print("\tReactive Power Generation :", qg)
        print("\t\tViolated Constraint :", q_min, "<=", qg, "<=", q_max)

Bus 1 :
	Reactive Power Generation : 17.126003190486458
		Violated Constraint : -5.0 <= 17.126003190486458 <= 15.0
Bus 12 :
	Reactive Power Generation : 244.12548647779846
		Violated Constraint : -35.0 <= 244.12548647779846 <= 120.0
Bus 15 :
	Reactive Power Generation : -39.82132378723966
		Violated Constraint : -10.0 <= -39.82132378723966 <= 30.0
Bus 19 :
	Reactive Power Generation : -9.862098238162522
		Violated Constraint : -8.0 <= -9.862098238162522 <= 24.0
Bus 34 :
	Reactive Power Generation : -21.42188338281757
		Violated Constraint : -8.0 <= -21.42188338281757 <= 24.0
Bus 55 :
	Reactive Power Generation : -15.709409836278402
		Violated Constraint : -8.0 <= -15.709409836278402 <= 23.0
Bus 56 :
	Reactive Power Generation : 17.66478039625501
		Violated Constraint : -8.0 <= 17.66478039625501 <= 15.0
Bus 74 :
	Reactive Power Generation : 11.145662417196487
		Violated Constraint : -6.0 <= 11.145662417196487 <= 9.0
Bus 76 :
	Reactive Power Generation : -17.92890513640784
		Violated Con

In [33]:
# The same is true for the voltage magnitudes at the same busses as before

for key in md.data['elements']['bus'].keys():
    vm = md.data['elements']['bus'][key]['vm']
    if not v_min <= vm <= v_max:
        print("Bus", key, ":")
        print("\tVoltage Magnitude :", vm)
        print("\t\tViolated Constraint :", v_min, "<=", vm, "<=", v_max)

Bus 2 :
	Voltage Magnitude : 0.8870470833038929
		Violated Constraint : 0.94 <= 0.8870470833038929 <= 1.06
Bus 20 :
	Voltage Magnitude : 0.9123340907590275
		Violated Constraint : 0.94 <= 0.9123340907590275 <= 1.06
Bus 21 :
	Voltage Magnitude : 0.8842757939964638
		Violated Constraint : 0.94 <= 0.8842757939964638 <= 1.06
Bus 22 :
	Voltage Magnitude : 0.9054834687457627
		Violated Constraint : 0.94 <= 0.9054834687457627 <= 1.06
Bus 51 :
	Voltage Magnitude : 0.9192213998291727
		Violated Constraint : 0.94 <= 0.9192213998291727 <= 1.06
Bus 52 :
	Voltage Magnitude : 0.908366500817184
		Violated Constraint : 0.94 <= 0.908366500817184 <= 1.06
Bus 53 :
	Voltage Magnitude : 0.9212238051691977
		Violated Constraint : 0.94 <= 0.9212238051691977 <= 1.06
Bus 58 :
	Voltage Magnitude : 0.9123965283692741
		Violated Constraint : 0.94 <= 0.9123965283692741 <= 1.06
Bus 86 :
	Voltage Magnitude : 0.9209025465882543
		Violated Constraint : 0.94 <= 0.9209025465882543 <= 1.06


In [34]:
# From here on out, I am just gathering the data to convert it into a dataframe

p_generated = [md.data['elements']['generator'][key]['pg']
                    for key in md.data['elements']['generator'].keys()]

q_generated = [md.data['elements']['generator'][key]['qg']
                    for key in md.data['elements']['generator'].keys()]

v_generated = [md.data['elements']['generator'][key]['vg']
                    for key in md.data['elements']['generator'].keys()]

vmg_at_bus = [md.data['elements']['bus'][key]['vm']
                    for key in md.data['elements']['bus'].keys()]
                    
voltage_angle = [md.data['elements']['bus'][key]['va']
                    for key in md.data['elements']['bus'].keys()]

In [35]:
data = np.concatenate((p_generated, q_generated, v_generated, vmg_at_bus, voltage_angle))

In [36]:
p_gen_cols = ["p_gen{}".format(i + 1) for i in range(54)]
q_gen_cols = ["q_gen{}".format(i + 1) for i in range(54)]
v_gen_cols = ["v_gen{}".format(i + 1) for i in range(54)]
v_at_bus_cols = ["v_at_bus{}".format(i + 1) for i in range(118)]
va_at_bus_cols = ["va_at_bus{}".format(i + 1) for i in range(118)]

cols = np.concatenate((p_gen_cols, q_gen_cols, v_gen_cols, v_at_bus_cols, va_at_bus_cols))

new_df = pd.DataFrame(data=[data], columns=cols)

In [37]:
new_df['time_to_solve'] = results['Solver'][0]['Time']

In [38]:
new_df

,p_gen1,p_gen2,p_gen3,p_gen4,p_gen5,p_gen6,p_gen7,p_gen8,p_gen9,p_gen10,...,va_at_bus110,va_at_bus111,va_at_bus112,va_at_bus113,va_at_bus114,va_at_bus115,va_at_bus116,va_at_bus117,va_at_bus118,time_to_solve
0,99.452442,17.099003,78.932786,13.149665,426.663473,100.677839,54.181558,37.167624,52.030772,2.98448,...,-18.454651,-17.742911,-18.137786,4.698496,3.54098,3.80597,21.728893,-1.256122,12.310784,0.134182
